In [1]:
import numpy as np
import cv2

**Užduotis** - apskaičiuoti IoU stačiakampiams ir Mask IoU kaukėms

Stačiakampiai formatu $(x_0, y_0, x_1, y_1)$

In [2]:
gt_bbox = (150, 200, 250, 300)
pred_bbox = (136, 211, 242, 309)

Kaukės numpy matricose (0 - kaukės nėra, 1 - kaukė yra)

In [3]:
gt_mask = np.zeros((100, 100), dtype = np.uint8)
_ = cv2.circle(gt_mask, (50, 50), 20, 1, -1)

pred_mask = np.zeros((100, 100), dtype = np.uint8)
_ = cv2.rectangle(pred_mask, (33, 33), (67, 67), 1, -1)

In [4]:
def resize_mask(mask, bbox, target_size=(128, 128)):
  x_start = int(bbox[0] * target_size[1])
  y_start = int(bbox[1] * target_size[0])
  x_end = int((bbox[0] + bbox[2]) * target_size[1])
  y_end = int((bbox[1] + bbox[3]) * target_size[0])

  return cv2.resize(mask, (x_end - x_start, y_end - y_start), interpolation=cv2.INTER_LINEAR)

In [5]:
gt_resized   = resize_mask(gt_mask,   gt_bbox,   (400,600))
pred_resized = resize_mask(pred_mask, pred_bbox, (400,600))


In [11]:
import torch

ys = gt_mask
y_hats = pred_mask

intersection = np.bitwise_and(ys, y_hats)
union = np.bitwise_or(ys, y_hats)

intersection = np.sum(intersection)
union = np.sum(union)

# Jaccard = || A \intersect B || / || A \union B ||
iou = intersection / union

iou

0.8398813936249073

In [8]:
def calculate_iou(box1, box2):
  x1, y1, x2, y2 = box1
  x3, y3, x4, y4 = box2

  inter_x1 = max(x1, x3)
  inter_y1 = max(y1, y3)
  inter_x2 = min(x2, x4)
  inter_y2 = min(y2, y4)

  if inter_x2 > inter_x1 and inter_y2 > inter_y1:
    intersection = (inter_x2 - inter_x1) * (inter_y2 - inter_y1)
  else:
    intersection = 0

  box1_area = (x2 - x1) * (y2 - y1)
  box2_area = (x4 - x3) * (y4 - y3)

  union = box1_area + box2_area - intersection

  # Jaccard = || A \intersect B || / || A \union B ||
  iou = intersection / union
  return iou

calculate_iou(gt_bbox, pred_bbox)

0.6711475409836065